# Chemical-Disease Relation (CDR) Tutorial

In this example, we'll be writing an application to extract *mentions of* **chemical-induced-disease relationships** from Pubmed abstracts, as per the [BioCreative CDR Challenge](http://www.biocreative.org/resources/corpora/biocreative-v-cdr-corpus/).  At core, we will be constructing a model to classify _candidate_ CDR mentions as either true or false.

## Part IV: Training a Model with Data Programming

In this part of the tutorial, we will train a statistical model to differentiate between true and false `ChemicalDisease` mentions.

We will train this model using _data programming_, and we will **ignore** the training labels provided with the training data. This is a more realistic scenario; in the wild, hand-labeled training data is rare and expensive. Data programming enables us to train a model using only a modest amount of hand-labeled data for validation and testing. For more information on data programming, see the [NIPS 2016 paper](https://arxiv.org/abs/1605.07723).

In [ ]:
%load_ext autoreload
%autoreload 2
%matplotlib inline

import numpy as np
from snorkel import SnorkelSession
session = SnorkelSession()

We repeat our definition of the `ChemicalDisease` `Candidate` subclass from Parts II and III.

In [ ]:
from snorkel.models import candidate_subclass

ChemicalDisease = candidate_subclass('ChemicalDisease', ['chemical', 'disease'])

## Loading `CandidateSet` objects

We reload the `CandidateSet` objects from the previous parts of the tutorial. Note that we will now process all three (training, validation, and test) as we go, because each plays a distinct role in Parts IV and V.

In [ ]:
from snorkel.models import CandidateSet

train = session.query(CandidateSet).filter(CandidateSet.name == 'CDR Training Candidates').one()
dev = session.query(CandidateSet).filter(CandidateSet.name == 'CDR Development Candidates').one()
test = session.query(CandidateSet).filter(CandidateSet.name == 'CDR Test Candidates').one()

candidate_sets = [train, dev, test]

## _TEMPORARY:_ We'll now create a small subsample to use

Create a small subset to proceed with:

In [ ]:
train_1K = CandidateSet(name='CDR Training Candidates 1K')
for candidate in train.candidates[:1000]:
    train_1K.append(candidate)
session.add(train_1K)
session.commit()

**OR** load if already created:

In [ ]:
train_1K = session.query(CandidateSet).filter(CandidateSet.name == 'CDR Training Candidates 1K').one()

## Automatically Creating Features
Recall that our goal is to distinguish between true and false mentions of chemical-disease relations. To train a model for this task, we first embed our `ChemicalDisease` candidates in a feature space.

In [ ]:
from snorkel.annotations import FeatureManager

feature_manager = FeatureManager()

We can create a new feature set:

**_NOTE: This may take ~1 hour currently_**

In [ ]:
%time train_features = feature_manager.create(session, train_1K, 'Train 1K Features')

**OR** if we've already created one, we can simply load as follows:

In [ ]:
%time F = feature_manager.load(session, train_1K, 'Train 1K Features')

Note that the returned matrix is a special subclass of the `scipy.sparse.csr_matrix` class, with some special features which we demonstrate below:

In [ ]:
F

In [ ]:
F.get_candidate(0)

In [ ]:
F.get_key(0)

## Creating Labeling Functions
Labeling functions are a core tool of data programming. They are heuristic functions that aim to classify candidates correctly. Their outputs will be automatically combined and denoised to estimate the probabilities of training labels for the training data.

In [ ]:
import random
def LF_test(candidate):
    return 1 if random.random() < 0.01 else 0
def LF_test_2(candidate):
    return 1 if random.random() < 0.1 else 0

We maintain a list of all LFs for convenience.

In [ ]:
LFs = [LF_test, LF_test_2]

## Applying Labeling Functions

First we construct a `CandidateLabeler`.

In [ ]:
from snorkel.annotations import LabelManager

label_manager = LabelManager()

Next we run the `CandidateLabeler` to to apply the labeling functions to the training `CandidateSet`.

In [ ]:
%time L = label_manager.create(session, train_1K, 'LF Labels', f=LFs)
L

**OR** load if we've already created:

In [ ]:
L = label_manager.load(session, train_1K, 'LF Labels')
L

We can also rerun a single labeling function (or more!) with the below command.

Do this to test changes to the labeling functions.

In [ ]:
def LF_test_3(candidate):
    return -1 if random.random() < 0.2 else 0

L = label_manager.update(session, train_1K, 'LF Labels', f=[LF_test_3])
L

We can view statistics about the resulting label matrix:

In [ ]:
L.lf_stats()

## Fitting the Generative Model
We estimate the accuracies of the labeling functions without supervision. Specifically, we estimate the parameters of a `NaiveBayes` generative model.

In [ ]:
from snorkel.learning import NaiveBayes

gen_model = NaiveBayes()
gen_model.train(L)

In [ ]:
gen_model.w

In [ ]:
gen_model.save(session, 'Generative Params 2')

In [ ]:
gen_model.load(session, 'Generative Params 2')
gen_model.w

We now apply the generative model to the training candidates.

In [ ]:
train_marginals = gen_model.marginals(L)

## Training the Discriminative Model
We use the estimated probabilites to train a discriminative model that classifies each `Candidate` as a true or false mention.

In [ ]:
from snorkel.learning import LogReg

disc_model = LogReg()
disc_model.train(F, train_marginals, n_iter=1500, rate=1e-5)

In [ ]:
disc_model.w.shape

In [ ]:
%time disc_model.save(session, "Discriminative Params 1")

In [ ]:
w_prev = disc_model.w
%time disc_model.load(session, "Discriminative Params 1")
np.all(disc_model.w == w_prev)

## Evaluating on the Development `CandidateSet`

In [ ]:
test_labels=[]
for candidate in sorted_test_candidates:
    test_labels.append(1 if candidate in gold_candidate_set else -1)
test_labels = np.asarray(test_labels)

score(sorted_test_candidates, test_labels, pred, gold_candidate_set, \
      train_marginals=train_marginals, test_marginals=test_marginals)

After evaluating on the development `CandidateSet`, the labeling functions can be modified. Try changing the labeling functions to improve performance. You can view the true positives, false positives, true negatives, and false negatives using the `Viewer`.

## Saving the Discriminative Model's Parameters
We save the model's parameters for use in Part V.

Next, in Part V, we will test our model on the test `CandidateSet`.